In [1]:
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers import UpSampling2D
from keras.layers import Conv2D
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing import image
import os, sys
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [ ]:
class DCGAN():
    def __init__(self):
        self.image_rows = 96
        self.image_cols = 96
        self.channels = 3
        self.image_shape = (self.image_rows, self.image_cols, self.channels)
        self.latent_dim = 100
        
        optimizer = Adam(0.0002, 0.5)
        optimizerD = RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)
        optimizerG = RMSprop(lr=0.0004, clipvalue=1.0, decay=6e-8)
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        self.generator = self.build_generator()
        
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        #对生成器进行训练即可
        self.discriminator.trainable = False
        
        #模型输入z，输出valid
        valid = self.discriminator(img)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):
        model = Sequential()
        model.add(Dense(512*6*6, activation='relu', input_dim=self.latent_dim))
        model.add(Reshape((6,6,512)))  #reshape成 6*6*512
        model.add(UpSampling2D()) #upsampling后成为 12*12*512
        model.add(Conv2D(256, kernel_size=5, padding='same'))  #转成12*12*256
        model.add(BatchNormalization(momentum=0.8)) #控制过拟合
        model.add(Activation('relu')) #激活函数relu
        model.add(UpSampling2D())  #24*24*256
        model.add(Conv2D(128, kernel_size=5, padding='same'))  #转成24*24*128
        model.add(BatchNormalization(momentum=0.8)) #控制过拟合
        model.add(Activation('relu')) #激活函数relu
        model.add(UpSampling2D())  #48*48*256
        model.add(Conv2D(64, kernel_size=5, padding='same'))  #转成48*48*64
        model.add(BatchNormalization(momentum=0.8)) #控制过拟合
        model.add(Activation('relu')) #激活函数relu
        model.add(UpSampling2D())  #96*96*64
        model.add(Conv2D(self.channels, kernel_size=5, padding='same'))  #转成96*96*3,即最后生成的图片size
        model.add(Activation('tanh'))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim, ))
        img = model(noise)
        return Model(noise, img)  #输入噪声，输出img
    
    def build_discriminator(self):
        depth = 32
        model = Sequential()
        model.add(Conv2D(64, kernel_size=5, strides=2, input_shape=self.image_shape, padding='same'))  #48*48*64
        model.add(LeakyReLU(alpha=0.2)) 
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))  #24*24*128
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=5, strides=2, padding="same")) #12*12*256
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(512, kernel_size=5, strides=1, padding="same"))  #6*6*512
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
        
        img = Input(shape=self.image_shape)
        score = model(img)
        
        return Model(img, score)
    
    def load_batch_images(self, batch_size, dirName):
        image_names = np.array(os.listdir(os.path.join(dirName)))
        index = np.random.randint(0, image_names.shape[0], batch_size)
        image_names = image_names[index]
        imgs = []
        
        for i in range(len(image_names)):
            img = image.load_img(os.path.join(dirName, image_names[i]), target_size=(96, 96))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            imgs.append(img)
            
        imgs = np.concatenate([img for img in imgs])
        imgs = imgs / 127.5 - 1 #正则化，[-1,1]
        return imgs
    
    def save_imgs(self, epoch):
        row = 5
        col = 5
        noise = np.random.normal(0, 1, (row*col, self.latent_dim))
        gen_imgs = self.generator.predict(noise) #此时数据[-1,1]范围内
        gen_imgs = 0.5*gen_imgs + 0.5 #恢复成[0, 1]
        
        fig, axs = plt.subplots(row, col, figsize = (10, 10))
        count = 0
        for i in range(row):
            for j in range(col):
                axs[i, j].imshow(gen_imgs[count, :, :, :])
                axs[i, j].axis('off')
                count += 1
        fig.savefig("images/epoch_%d.png" %epoch)
        plt.close()
        
    
    def train(self, epochs, batch_size=256, save_steps=50):
        true_img = np.ones((batch_size, 1))
        fake_img = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            imgs = self.load_batch_images(batch_size, 'cartoonFaces')
            noise = np.random.normal(0,1,(batch_size, self.latent_dim))
            fake_imgs = self.generator.predict(noise)
            
            d_loss_real = self.discriminator.train_on_batch(imgs, true_img)
            d_loss_fake = self.discriminator.train_on_batch(fake_imgs, fake_img)
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_fake)
            
            g_loss = self.combined.train_on_batch(noise, true_img)
            
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            if epoch % save_steps == 0:
                self.combined.save('./model/combined_model_%d.h5' %epoch)
                self.discriminator.save('./model/discriminator_model_%d.h5' %epoch)
                self.save_imgs(epoch)

In [ ]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=50000, save_steps=20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 24, 24, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 128)       0         
__________

/root/anaconda3/envs/xuhui/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 8.898353, acc.: 0.00%] [G loss: 0.705413]
1 [D loss: 2.387619, acc.: 8.98%] [G loss: 2.504880]
2 [D loss: 0.224674, acc.: 92.19%] [G loss: 0.076950]
3 [D loss: 0.002385, acc.: 100.00%] [G loss: 0.026370]
4 [D loss: 0.077050, acc.: 97.27%] [G loss: 0.046905]
5 [D loss: 0.828986, acc.: 61.33%] [G loss: 15.439524]
6 [D loss: 0.000000, acc.: 100.00%] [G loss: 5.322999]
7 [D loss: 1.105914, acc.: 74.61%] [G loss: 0.061565]
8 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.319875]
9 [D loss: 0.000012, acc.: 100.00%] [G loss: 0.000364]
10 [D loss: 1.091919, acc.: 61.33%] [G loss: 15.118797]
11 [D loss: 0.000000, acc.: 100.00%] [G loss: 13.879553]
12 [D loss: 0.000006, acc.: 100.00%] [G loss: 2.401495]
13 [D loss: 0.024386, acc.: 99.22%] [G loss: 0.168480]
14 [D loss: 0.005155, acc.: 100.00%] [G loss: 0.402643]
15 [D loss: 0.012435, acc.: 100.00%] [G loss: 0.645479]
16 [D loss: 0.136698, acc.: 96.09%] [G loss: 2.488690]
17 [D loss: 5.209022, acc.: 1.95%] [G loss: 16.118101]
18 [D loss:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


21 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
22 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
23 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
24 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
25 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
26 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
27 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
28 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
29 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
30 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
31 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
32 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
33 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
34 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
35 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
36 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
37 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118101]
38 [D loss: 0.000000, acc.: 100

...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


14761 [D loss: 0.006112, acc.: 100.00%] [G loss: 12.518354]
14762 [D loss: 0.078445, acc.: 96.09%] [G loss: 13.137299]
14763 [D loss: 0.003505, acc.: 100.00%] [G loss: 11.747458]
14764 [D loss: 0.050480, acc.: 98.05%] [G loss: 10.408795]
14765 [D loss: 0.058227, acc.: 98.83%] [G loss: 10.827123]
14766 [D loss: 0.030742, acc.: 99.61%] [G loss: 8.158533]
14767 [D loss: 0.116779, acc.: 95.31%] [G loss: 10.511171]
14768 [D loss: 0.019226, acc.: 99.22%] [G loss: 10.190865]
14769 [D loss: 0.041118, acc.: 98.44%] [G loss: 9.644522]
14770 [D loss: 0.037633, acc.: 98.83%] [G loss: 9.385301]
14771 [D loss: 0.024997, acc.: 99.22%] [G loss: 8.432734]
14772 [D loss: 0.063885, acc.: 97.27%] [G loss: 10.188462]
14773 [D loss: 0.013073, acc.: 100.00%] [G loss: 9.025021]
14774 [D loss: 0.092619, acc.: 96.09%] [G loss: 11.281576]
14775 [D loss: 0.001905, acc.: 100.00%] [G loss: 9.408563]
14776 [D loss: 0.194765, acc.: 89.84%] [G loss: 13.893971]
14777 [D loss: 0.000352, acc.: 100.00%] [G loss: 12.491681

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


14781 [D loss: 0.037058, acc.: 98.83%] [G loss: 8.562515]
14782 [D loss: 0.043365, acc.: 98.44%] [G loss: 7.855968]
14783 [D loss: 0.031782, acc.: 99.22%] [G loss: 7.066537]
14784 [D loss: 0.065128, acc.: 97.66%] [G loss: 5.837844]
14785 [D loss: 0.021113, acc.: 99.22%] [G loss: 5.647801]
14786 [D loss: 0.103553, acc.: 96.88%] [G loss: 6.682539]
14787 [D loss: 0.028373, acc.: 99.22%] [G loss: 5.697779]
14788 [D loss: 0.164664, acc.: 92.97%] [G loss: 10.435436]
14789 [D loss: 0.002181, acc.: 100.00%] [G loss: 6.100872]
14790 [D loss: 0.223091, acc.: 90.62%] [G loss: 11.140458]
14791 [D loss: 0.012397, acc.: 100.00%] [G loss: 9.482956]
14792 [D loss: 0.199312, acc.: 91.41%] [G loss: 12.988586]
14793 [D loss: 0.000330, acc.: 100.00%] [G loss: 13.591349]
14794 [D loss: 0.001631, acc.: 100.00%] [G loss: 10.644651]
14795 [D loss: 0.112341, acc.: 96.88%] [G loss: 10.985592]
14796 [D loss: 0.001619, acc.: 100.00%] [G loss: 10.706945]
14797 [D loss: 0.008438, acc.: 99.61%] [G loss: 8.044179]
14

14921 [D loss: 0.020061, acc.: 99.22%] [G loss: 11.191043]
14922 [D loss: 0.007997, acc.: 100.00%] [G loss: 9.763490]
14923 [D loss: 0.072278, acc.: 97.66%] [G loss: 10.300646]
14924 [D loss: 0.036139, acc.: 98.83%] [G loss: 10.256315]
14925 [D loss: 0.007150, acc.: 100.00%] [G loss: 9.703998]
14926 [D loss: 0.139230, acc.: 95.70%] [G loss: 11.366230]
14927 [D loss: 0.040007, acc.: 98.83%] [G loss: 11.055935]
14928 [D loss: 0.026904, acc.: 99.61%] [G loss: 8.654642]
14929 [D loss: 0.262819, acc.: 87.89%] [G loss: 15.218399]
14930 [D loss: 0.020925, acc.: 99.22%] [G loss: 8.776030]
14931 [D loss: 0.350608, acc.: 83.20%] [G loss: 15.083771]
14932 [D loss: 0.000007, acc.: 100.00%] [G loss: 15.210420]
14933 [D loss: 0.004104, acc.: 100.00%] [G loss: 10.701661]
14934 [D loss: 0.283435, acc.: 89.06%] [G loss: 14.884054]
14935 [D loss: 0.000358, acc.: 100.00%] [G loss: 15.056580]
14936 [D loss: 0.000797, acc.: 100.00%] [G loss: 13.867352]
14937 [D loss: 0.006644, acc.: 100.00%] [G loss: 12.89